# Generalizations of Max-Flow

The purpose of this assignment is to learn about the min-cost flow problem, a generalization of max-flow, and to familiarize yourself with implementing and solving linear programs.

## Min-Cost Flow

Recall that a flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$. In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. (Note that this input generalizes the input to two important problems we discussed so far: in max flow, the edge weights were not important while in shortest paths, the edge capacities were not important.) 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

### Question 1

Answer Problem 1 in HW4-theoretical.

### Question 2

To implement your reduction from Problem 1 in HW4-theoretical, you will work with some standard benchmark instances for min-cost flow found [here](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/index.html). The format of the data is described in the [Info](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/info) file. You are to read in the graph from the data file in a form that can be solved using NetworkX's `min_cost_flow` function. Note that the data sometimes lists multiple edges between the same nodes, but with different costs or capacities. In forming the graph, you need to implement your reduction from the previous question and form a `DiGraph` instance, because the `min_cost_flow` function cannot handle multi-edges, even though the package offers `MultiDiGraph` objects.

In [7]:
import networkx as nx

def create_graph(infile):
    G=nx.DiGraph() 
    node_dict = {}
    with open(infile, 'r') as f:
        for line in f:
            s=line.strip().split(' ')
            if (s[0] == "n") :
                node = s[1]
                demand = s[2]
                node_dict[node] = int(demand)
                G.add_nodes_from(node_dict)
                for node in G.node:
                    G.node[node]['demand']=node_dict[node]
    with open(infile, 'r') as f:
        for line in f:
            s = line.strip().split(" ")
            if(s[0] == "a"):
                tail = s[1]
                head = s[2]
                G.add_edge(tail,head)
                capacity = s[4]
                cost = s[5]
                G.edge[tail][head]['capacity'] = int(capacity)
                G.edge[tail][head]['weight'] = int(cost)
                for node in G.node:
                    if G.node[tail] == {}:
                        G.node[tail]['demand'] = 0
    #get duplicate edges
    with open(infile, 'r') as f:
        for line in f:
            s = line.strip().split(" ")
            if(s[0] == "a"):
                tail = s[1]
                head = s[2]
                capacity = s[4]
                cost = s[5]
                if (tail,head) in G.edges() and int(cost) != G.edge[tail][head]['weight']:
                    extra_node = tail + 'to' + head
                    G.add_node(extra_node)
                    G.node[extra_node]['demand'] = 0
                    G.add_edge(tail,extra_node)
                    G.add_edge(extra_node,head)
                    G.edge[tail][extra_node]['capacity'] = int(capacity)
                    G.edge[tail][extra_node]['weight'] = float(cost)/2
                    G.edge[extra_node][head]['capacity'] = int(capacity)
                    G.edge[extra_node][head]['weight'] = float(cost)/2
    return G

The following will check that your code outputs the expected min cost flow values on several test instances.

In [8]:
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')

print "Correct value for _40 instance:", nx.min_cost_flow_cost(G_40) == 52099553858
print "Correct value for _6830 instance:", nx.min_cost_flow_cost(G_6830) == 299390431788
print "Correct value for _176280 instance:", nx.min_cost_flow_cost(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


## Linear Programming

Instead of using special-purpose min-cost flow solvers, you will now formulate the problems as linear programs and use general-purpose LP solvers to find the solutions.

### Question 3

Implement the following function to formulate the flow LP and return the optimal value (i.e., minimum cost over feasible flows).

In [9]:
import pulp

def lp_flow_value(G):

# Adding in problem data from our graph
    demand = nx.get_node_attributes(G,'demand')
    weight = nx.get_edge_attributes(G,'weight')
    capacity = nx.get_edge_attributes(G,'capacity')
    edges = G.edges()
    nodes = G.nodes()

# define flow
    flow = pulp.LpVariable.dicts("Route",edges,0,None,pulp.LpInteger)

#Container for problem
    prob = pulp.LpProblem("Min Cost Flow",pulp.LpMinimize)
# Creates the objective function
    prob += pulp.lpSum([weight[e]* flow[e] for e in edges]), "Total Cost of Flow"

    for e in edges:
        flow[e].bounds(0, capacity[e])

    for n in nodes:
        prob += (pulp.lpSum([flow[(i,j)] for (i,j) in edges if j == n])) - (pulp.lpSum([flow[(i,j)] for (i,j) in edges if i == n])) == demand[n]
    
    prob.writeLP("simple_MCFP.lp")

    prob.solve()
    
    return int(pulp.value(prob.objective))

The following will check that the LP finds the same optimal values as previously.

In [10]:
print "Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858
print "Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788
print "Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
